In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [11]:
from datasets import load_dataset

test_set = load_dataset('csv', data_files={
            'train': "/workspace/expl/ECQA-Dataset/cqa_data_test.csv"})

Using custom data configuration default-668447907e70c92a
Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-668447907e70c92a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
option_keys = sorted([
    key for key in test_set['train'][0]
    if key.startswith("q_op")
    ])


input_strs = []
target_strs = []    

ans_type = 'taskA_pos' 
for example in test_set['train']:

    include_expl = True
    ans_type = 'taskA_pos' # or 'taskA_pos', 'taskA_neg', 'taskB'

    all_options = "".join([f"- {j+1}. {example[option_key]}\n" for j, option_key in enumerate(option_keys)])
    input_str = f"Question: {example['q_text']}\n{all_options}\nAnswer:\n"
    target_str = f"{example['q_ans']}. {example[ans_type]} <|endoftext|>"
    input_strs.append(input_str)
    target_strs.append(target_str)


In [13]:
model = AutoModelForCausalLM.from_pretrained("/workspace/quality/result/answer/checkpoint-3100")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [20]:
import torch

for x in input_strs:
    x = tokenizer(x, return_tensors='pt')['input_ids']
    print(x)
    with torch.no_grad():
        result = model.generate(
                    x, 
                    max_length=100, 
                    num_beams=3, 
                    early_stopping=True
                )
    sel_output = tokenizer.decode(result[0], skip_special_tokens=True)
    break

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[24361,    25,  1867,  1611,   286,   661,   481,  2130,   508,   318,
           287,  3877,   286,  1628,   804,   329,    30,   198,    12,   352,
            13, 31828,   198,    12,   362,    13,  5490,   198,    12,   513,
            13,   423,  3725,   198,    12,   604,    13,  1011,  3877,   198,
            12,   642,    13,  6478,   198,   198, 33706,    25,   198]])
